In [ ]:
import numpy as np


In [ ]:
datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

In [ ]:
print(datain)
print(datain.shape)   # Useful if it’s an array
print(type(datain)) 

In [ ]:
print(dataout)
print(dataout.shape)   # Useful if it’s an array
print(type(dataout)) 

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize

# --- Initial data ---
X_init = np.array([
    [0.66579958, 0.12396913],
    [0.87779099, 0.7786275 ],
    [0.14269907, 0.34900513],
    [0.84527543, 0.71112027],
    [0.45464714, 0.29045518],
    [0.57771284, 0.77197318],
    [0.43816606, 0.68501826],
    [0.34174959, 0.02869772],
    [0.33864816, 0.21386725],
    [0.70263656, 0.9265642 ]
])

y_init = np.array([0.53899612, 0.42058624, -0.06562362, 0.29399291, 
                   0.21496451, 0.02310555, 0.24461934, 0.03874902, 
                   -0.01385762, 0.61120522])

In [ ]:
# Add the new observation
X_new = np.array([[0.365139, 0.474667]])
y_new = np.array([-0.006080823079048435])

# Combine with previous data
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])

X_init = X_all
y_init = y_all

In [ ]:
print(X_init)
print(X_init.shape)   # Useful if it’s an array
print(type(X_init)) 

In [ ]:
print(y_init)
print(y_init.shape)   # Useful if it’s an array
print(type(y_init)) 

# Week 1 

In [ ]:


# --- Fit a Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_init)

# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    sigma = sigma[0]
    mu = mu[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    from scipy.stats import norm
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative because we will minimize

# --- Optimize acquisition function ---
y_best = y_init.max()
bounds = [(0,1), (0,1)]

res = minimize(lambda x: expected_improvement(x, gp, y_best),
               x0=np.random.rand(2),
               bounds=bounds,
               method='L-BFGS-B')

x_next = res.x
print("Next point to evaluate:", x_next)


# Week 2

In [ ]:
# slightly increase alpha (the noise level) since the function outputs are noisy
# --- Fit a Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-3, normalize_y=True)
gp.fit(X_init, y_init)

In [ ]:
# --- Expected Improvement acquisition function ---
# change xi to 0.01
def expected_improvement(x, gp, y_best, xi=0.05):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    sigma = sigma[0]
    mu = mu[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    from scipy.stats import norm
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative because we will minimize

In [ ]:
from scipy.optimize import minimize
import numpy as np

y_best = y_all.max()
bounds = [(0, 1), (0, 1)]

def propose_location(acquisition, gp, y_best, bounds, n_restarts=25):
    best_x = None
    best_acq = 1e10
    for i in range(n_restarts):
        x0 = np.random.rand(2)
        res = minimize(lambda x: acquisition(x, gp, y_best),
                       x0=x0,
                       bounds=bounds,
                       method='L-BFGS-B')
        if res.fun < best_acq:
            best_acq = res.fun
            best_x = res.x
    return best_x

x_next = propose_location(expected_improvement, gp, y_best, bounds)
print("Next point to evaluate:", x_next)


# Week 3

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Initial data ---
X_init = np.array([
    [0.66579958, 0.12396913],
    [0.87779099, 0.7786275 ],
    [0.14269907, 0.34900513],
    [0.84527543, 0.71112027],
    [0.45464714, 0.29045518],
    [0.57771284, 0.77197318],
    [0.43816606, 0.68501826],
    [0.34174959, 0.02869772],
    [0.33864816, 0.21386725],
    [0.70263656, 0.9265642 ]
])

y_init = np.array([0.53899612, 0.42058624, -0.06562362, 0.29399291, 
                   0.21496451, 0.02310555, 0.24461934, 0.03874902, 
                   -0.01385762, 0.61120522])

# --- Add new observations (example: last evaluated points) ---
X_new = np.array([
    [0.365139, 0.474667],
    [0.511479, 0.507777]
])
y_new = np.array([-0.006080823079048435, 0.6741068207629037])

# Combine all data
X_all = np.vstack((X_init, X_new))
y_all = np.hstack((y_init, y_new))

# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-3, normalize_y=True)
gp.fit(X_all, y_all)

# --- Expected Improvement ---
def expected_improvement(x, gp, y_best, xi=0.05):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu[0]
    sigma = sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative because we will minimize

# --- Acquisition optimizer with multiple restarts ---
def propose_location(acquisition, gp, y_best, bounds, n_restarts=25):
    best_x = None
    best_acq = 1e10
    for _ in range(n_restarts):
        x0 = np.random.rand(2)
        res = minimize(lambda x: acquisition(x, gp, y_best),
                       x0=x0,
                       bounds=bounds,
                       method='L-BFGS-B')
        if res.fun < best_acq:
            best_acq = res.fun
            best_x = res.x
    return best_x

# --- Select next point ---
y_best = y_all.max()
bounds = [(0,1), (0,1)]
x_next = propose_location(expected_improvement, gp, y_best, bounds)

print("Next point to evaluate:", x_next)

# Print with 6 decimal places
x_next_rounded = np.round(x_next, 6)
print("Next point to evaluate (6 decimal places):", x_next_rounded)


# Week 4

New point: [0.242364, 0.231669]
Output: -0.014065
This new point is far from the current best in input space (closer to the bottom-left corner).
Its negative output confirms that this region is not promising.
The GP surrogate model will now learn low predicted values in this area and also reduce uncertainty near this point.
✅ So this observation primarily helps exploration: it teaches the model where not to sample, which reduces wasted evaluations in low-value regions.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- All observed data points ---
X_all = np.array([
    [0.66579958, 0.12396913],
    [0.87779099, 0.7786275 ],
    [0.14269907, 0.34900513],
    [0.84527543, 0.71112027],
    [0.45464714, 0.29045518],
    [0.57771284, 0.77197318],
    [0.43816606, 0.68501826],
    [0.34174959, 0.02869772],
    [0.33864816, 0.21386725],
    [0.70263656, 0.9265642 ],
    [0.365139, 0.474667],
    [0.511479, 0.507777],
    [0.242364, 0.231669]  # Latest observation
])

y_all = np.array([
    0.53899612, 0.42058624, -0.06562362, 0.29399291, 
    0.21496451, 0.02310555, 0.24461934, 0.03874902, 
    -0.01385762, 0.61120522,
    -0.006080823079048435,
    0.6741068207629037,
    -0.014065043301466521  # Latest output
])

# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-3, normalize_y=True)
gp.fit(X_all, y_all)

# --- Expected Improvement ---
def expected_improvement(x, gp, y_best, xi=0.05):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu[0]
    sigma = sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative because we will minimize

# --- Acquisition optimizer with multiple random restarts ---
def propose_location(acquisition, gp, y_best, bounds, n_restarts=25):
    best_x = None
    best_acq = 1e10
    for _ in range(n_restarts):
        x0 = np.random.rand(2)
        res = minimize(lambda x: acquisition(x, gp, y_best),
                       x0=x0,
                       bounds=bounds,
                       method='L-BFGS-B')
        if res.fun < best_acq:
            best_acq = res.fun
            best_x = res.x
    return best_x

# --- Select next point ---
y_best = y_all.max()  # 0.6741068
bounds = [(0,1), (0,1)]
x_next = propose_location(expected_improvement, gp, y_best, bounds)

# Print next point to 6 decimal places
x_next_rounded = np.round(x_next, 6)
print("Next point to evaluate (6 decimal places):", x_next_rounded)


# Week 5
- The new y ≈ 0.0185 is small positive — better than many negative samples but far below your best (0.6741).
- This is useful exploration data: it helps the GP rule out parts of the space that are unlikely to contain the global maximum.
- 2 options:
- Exploit / refine around the current best — use EI with a small xi (e.g. xi = 0.01) to fine-tune near the highest region and try to improve the best value.
- Explore a nearby promising region — use EI with a larger xi (e.g. xi = 0.2) or UCB to push into uncertain regions (for example near [0.438,0.685] or between [0.438,0.685] and [0.511,0.507]) and check for another peak.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- All observed data points ---
X_all = np.array([
    [0.66579958, 0.12396913],
    [0.87779099, 0.7786275 ],
    [0.14269907, 0.34900513],
    [0.84527543, 0.71112027],
    [0.45464714, 0.29045518],
    [0.57771284, 0.77197318],
    [0.43816606, 0.68501826],
    [0.34174959, 0.02869772],
    [0.33864816, 0.21386725],
    [0.70263656, 0.9265642 ],
    [0.365139, 0.474667],
    [0.511479, 0.507777],
    [0.242364, 0.231669],
    [0.199598, 0.656821] # Latest observation
])

y_all = np.array([
    0.53899612, 0.42058624, -0.06562362, 0.29399291, 
    0.21496451, 0.02310555, 0.24461934, 0.03874902, 
    -0.01385762, 0.61120522,
    -0.006080823079048435,
    0.6741068207629037,
    -0.014065043301466521,
    0.018456695795070272 # Latest output
])

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- X_all, y_all should already include the latest point [0.199598,0.656821] -> 0.018456695795070272

kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-3, normalize_y=True)
gp.fit(X_all, y_all)

def expected_improvement(x, gp, y_best, xi):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu[0]; sigma = sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei

def propose_location(acquisition, gp, y_best, bounds, xi, n_restarts=40):
    best_x = None
    best_acq = 1e10
    for _ in range(n_restarts):
        x0 = np.random.rand(2)
        res = minimize(lambda x: acquisition(x, gp, y_best, xi),
                       x0=x0,
                       bounds=bounds,
                       method='L-BFGS-B')
        if res.success and res.fun < best_acq:
            best_acq = res.fun
            best_x = res.x
    return best_x

y_best = y_all.max()
bounds = [(0,1),(0,1)]

# Exploit candidate (low xi)
x_exploit = propose_location(expected_improvement, gp, y_best, bounds, xi=0.01, n_restarts=50)

# Explore candidate (higher xi)
x_explore = propose_location(expected_improvement, gp, y_best, bounds, xi=0.20, n_restarts=50)

# Round to 6 decimals
print("Exploit (6 d.p.):", np.round(x_exploit, 6))
print("Explore (6 d.p.):", np.round(x_explore, 6))


- With only one evaluation, pick a point that maximises the chance of improving the current best (0.6741 at [0.511479, 0.507777]) — i.e. a local, high-confidence exploit pick rather than broad exploration.
- Going with Exploit (6 d.p.): [0.425763 0.235879]

# Week 6
- The new y ≈ 0.12647 is better than the negative samples nearby but worse than the nearest moderately-good neighbor (0.21496 at [0.4546,0.2905]).
- Being close (dist ≈ 0.062) to that moderately-good point means this neighborhood is partially promising but not a hot spot compared with the global best.
- The observation will reduce posterior uncertainty around [0.43,0.24] and push the GP mean to reflect a moderate value in that region. - In practical terms, the GP is less likely to re-sample that exact spot and will prefer either (a) fine-tuning near the global best, or (b) searching nearby uncertain regions that could connect moderate regions to the best.
- Conclusion: this point helps rule out the bottom-right of the moderate cluster as the global maximizer — it’s useful negative evidence for large gains there.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- All observed data points ---
X_all = np.array([
    [0.66579958, 0.12396913],
    [0.87779099, 0.7786275 ],
    [0.14269907, 0.34900513],
    [0.84527543, 0.71112027],
    [0.45464714, 0.29045518],
    [0.57771284, 0.77197318],
    [0.43816606, 0.68501826],
    [0.34174959, 0.02869772],
    [0.33864816, 0.21386725],
    [0.70263656, 0.9265642 ],
    [0.365139, 0.474667],
    [0.511479, 0.507777],
    [0.242364, 0.231669],
    [0.199598, 0.656821],
    [0.425763, 0.235879] # Latest observation
])

y_all = np.array([
    0.53899612, 0.42058624, -0.06562362, 0.29399291, 
    0.21496451, 0.02310555, 0.24461934, 0.03874902, 
    -0.01385762, 0.61120522,
    -0.006080823079048435,
    0.6741068207629037,
    -0.014065043301466521,
    0.018456695795070272,
    0.12646677030059522 # Latest output
])

- Exploit. Use EI with a small xi (e.g. xi=0.01) and many restarts, seeded with the current best, to produce a local-improvement candidate near [0.511479,0.507777].
- That has the highest chance to raise maximum.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- assume X_all, y_all already include the latest point [0.425763,0.235879] -> 0.12646677030059522

kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-3, normalize_y=True)
gp.fit(X_all, y_all)

def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu[0]; sigma = sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei

def propose_exploit(gp, y_best, bounds=[(0,1),(0,1)], n_restarts=200):
    best_x = None
    best_acq = 1e10
    starts = [np.array([0.511479, 0.507777])]  # current best
    starts += [np.random.rand(2) for _ in range(n_restarts-1)]
    for x0 in starts:
        res = minimize(lambda x: expected_improvement(x, gp, y_best, xi=0.01),
                       x0=x0, bounds=bounds, method='L-BFGS-B')
        if res.success and res.fun < best_acq:
            best_acq = res.fun
            best_x = res.x
    return best_x

y_best = y_all.max()
x_next = propose_exploit(gp, y_best, n_restarts=200)
print("Exploit next (6 d.p.):", np.round(x_next, 6))


# Week 7
New point: [0.517484, 0.513827] → y = 0.5750720845426626.
This is high and close to current best (0.6741068 at [0.511479, 0.507777]), but not a new maximum.
It confirms a strong peak in that region — the GP will now be more confident there (lower σ) and predict high mean nearby.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- All observed data points ---
X_all = np.array([
    [0.66579958, 0.12396913],
    [0.87779099, 0.7786275 ],
    [0.14269907, 0.34900513],
    [0.84527543, 0.71112027],
    [0.45464714, 0.29045518],
    [0.57771284, 0.77197318],
    [0.43816606, 0.68501826],
    [0.34174959, 0.02869772],
    [0.33864816, 0.21386725],
    [0.70263656, 0.9265642 ],
    [0.365139, 0.474667],
    [0.511479, 0.507777],
    [0.242364, 0.231669],
    [0.199598, 0.656821],
    [0.425763, 0.235879],
    [0.517484, 0.513827] # Latest observation
])

y_all = np.array([
    0.53899612, 0.42058624, -0.06562362, 0.29399291, 
    0.21496451, 0.02310555, 0.24461934, 0.03874902, 
    -0.01385762, 0.61120522,
    -0.006080823079048435,
    0.6741068207629037,
    -0.014065043301466521,
    0.018456695795070272,
    0.12646677030059522, 
    0.5750720845426626 # Latest output
])

Do a local refinement around the best point. Run EI with a small xi (0.01 or 0.005), seed starts with the current best [0.511479, 0.507777] and include the new point as a seed. If optimization returns a point almost identical to the best (within ~1e-3), return a small jittered point around the best (Gaussian jitter scale ~0.01 clipped to [0,1]) to explore the immediate neighborhood.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# assume X_all, y_all exist and contain previous points

# --- Fit GP ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-3, normalize_y=True)
gp.fit(X_all, y_all)

# --- Expected Improvement (small xi for exploitation) ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu[0]; sigma = sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei

# --- Propose exploit: many restarts, seed current best & new point ---
def propose_exploit(gp, y_best, best_seed, new_seed, bounds=[(0,1),(0,1)], n_restarts=200):
    best_x = None
    best_acq = 1e10
    starts = [np.array(best_seed), np.array(new_seed)] + [np.random.rand(2) for _ in range(n_restarts-2)]
    for x0 in starts:
        res = minimize(lambda x: expected_improvement(x, gp, y_best, xi=0.01),
                       x0=x0, bounds=bounds, method='L-BFGS-B')
        if res.success and res.fun < best_acq:
            best_acq = res.fun
            best_x = res.x
    # if result is almost identical to best_seed, jitter slightly
    if np.linalg.norm(best_x - np.array(best_seed)) < 1e-3:
        jitter = np.random.normal(scale=0.01, size=2)
        best_x = np.clip(np.array(best_seed) + jitter, 0.0, 1.0)
    return best_x

y_best = y_all.max()
best_seed = [0.511479, 0.507777]   # prior best
new_seed = [0.517484, 0.513827]    # latest point

x_next = propose_exploit(gp, y_best, best_seed, new_seed, n_restarts=200)
print("Next exploit (6 d.p.):", np.round(x_next, 6))


If I still don’t exceed 0.6741 after a couple of local refinements, then switch to exploration (increase xi or try UCB or sample the other high-value cluster) because the local peak may be exhausted or noise-limited.

# Week 8
- Confirms a high-value region around ~[0.51,0.51].
- With three nearby high observations, posterior variance (σ) around ~[0.51,0.52] will be low. Expected Improvement (EI) will therefore require tighter, local improvements to beat the best; the acquisition surface tends to produce candidates that are either tiny refinements near the best or else search elsewhere for high uncertainty.
- EI with a small xi will exploit (fine-tune) around the [0.51,0.51] region.
- to find other peaks, I must force exploration (increase xi, use UCB, or occasionally sample a random high-uncertainty point).

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- All observed data points ---
X_all = np.array([
    [0.66579958, 0.12396913],
    [0.87779099, 0.7786275 ],
    [0.14269907, 0.34900513],
    [0.84527543, 0.71112027],
    [0.45464714, 0.29045518],
    [0.57771284, 0.77197318],
    [0.43816606, 0.68501826],
    [0.34174959, 0.02869772],
    [0.33864816, 0.21386725],
    [0.70263656, 0.9265642 ],
    [0.365139, 0.474667],
    [0.511479, 0.507777],
    [0.242364, 0.231669],
    [0.199598, 0.656821],
    [0.425763, 0.235879],
    [0.517484, 0.513827],
    [0.521491, 0.517865] # Latest observation
])

y_all = np.array([
    0.53899612, 0.42058624, -0.06562362, 0.29399291, 
    0.21496451, 0.02310555, 0.24461934, 0.03874902, 
    -0.01385762, 0.61120522,
    -0.006080823079048435,
    0.6741068207629037,
    -0.014065043301466521,
    0.018456695795070272,
    0.12646677030059522, 
    0.5750720845426626,
    0.5531590730253101 # Latest output
])

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm


# --- Fit GP ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-3, normalize_y=True)
gp.fit(X_all, y_all)

# --- Expected Improvement (small xi for exploitation) ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu[0]; sigma = sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei

# --- Propose local exploit: seed with best and latest point ---
def propose_local_exploit(gp, y_best, seeds, bounds=[(0,1),(0,1)], n_restarts=150):
    best_x = None
    best_acq = 1e10
    starts = seeds + [np.random.rand(2) for _ in range(n_restarts - len(seeds))]
    for x0 in starts:
        res = minimize(lambda x: expected_improvement(x, gp, y_best, xi=0.01),
                       x0=x0, bounds=bounds, method='L-BFGS-B')
        if res.success and res.fun < best_acq:
            best_acq = res.fun
            best_x = res.x
    # if optimizer returns point nearly identical to best seed, jitter slightly:
    if np.linalg.norm(best_x - seeds[0]) < 1e-3:
        print("jitterbug")
        best_x = np.clip(seeds[0] + np.random.normal(scale=0.008, size=2), 0.0, 1.0)
    return best_x

y_best = y_all.max()
best_seed = np.array([0.511479, 0.507777])   # prior best
new_seed  = np.array([0.521491, 0.517865])   # latest point

x_next = propose_local_exploit(gp, y_best, seeds=[best_seed, new_seed], n_restarts=150)
print("Exploit next (6 d.p.):", np.round(x_next, 6))


In [ ]:
# Week 9

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- All observed data points ---
X_all = np.array([
    [0.66579958, 0.12396913],
    [0.87779099, 0.7786275 ],
    [0.14269907, 0.34900513],
    [0.84527543, 0.71112027],
    [0.45464714, 0.29045518],
    [0.57771284, 0.77197318],
    [0.43816606, 0.68501826],
    [0.34174959, 0.02869772],
    [0.33864816, 0.21386725],
    [0.70263656, 0.9265642 ],
    [0.365139, 0.474667],
    [0.511479, 0.507777],
    [0.242364, 0.231669],
    [0.199598, 0.656821],
    [0.425763, 0.235879],
    [0.517484, 0.513827],
    [0.521491, 0.517865],
    [0.529509, 0.525943] # Latest observation
])

y_all = np.array([
    0.53899612, 0.42058624, -0.06562362, 0.29399291, 
    0.21496451, 0.02310555, 0.24461934, 0.03874902, 
    -0.01385762, 0.61120522,
    -0.006080823079048435,
    0.6741068207629037,
    -0.014065043301466521,
    0.018456695795070272,
    0.12646677030059522, 
    0.5750720845426626,
    0.5531590730253101,
    0.6221952072714012 # Latest output
])

# Week 9
- New observation: [0.529509, 0.525943] → y = 0.6221952072714012.
- This is the second-best observation so far (behind 0.6741068 at [0.511479,0.507777]).
- It strengthens the view that the global maximum lies in the ~[0.51,0.53] × [0.51,0.53] 

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Fit GP ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-3, normalize_y=True)
gp.fit(X_all, y_all)

# --- Expected Improvement (small xi for exploitation) ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu[0]; sigma = sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei

# --- Propose exploit: seed with best and recent highs ---
def propose_exploit(gp, y_best, seeds, bounds=[(0,1),(0,1)], n_restarts=200):
    best_x = None
    best_acq = 1e10
    starts = seeds + [np.random.rand(2) for _ in range(n_restarts - len(seeds))]
    for x0 in starts:
        res = minimize(lambda x: expected_improvement(x, gp, y_best, xi=0.01),
                       x0=x0, bounds=bounds, method='L-BFGS-B')
        if res.success and res.fun < best_acq:
            best_acq = res.fun
            best_x = res.x
    # small jitter if returned point is nearly identical to first seed
    if np.linalg.norm(best_x - seeds[0]) < 1e-3:
        best_x = np.clip(seeds[0] + np.random.normal(scale=0.008, size=2), 0, 1)
    return best_x

y_best = y_all.max()
seeds = [np.array([0.511479, 0.507777]), np.array([0.521491, 0.517865]), np.array([0.517484, 0.513827])]
x_next = propose_exploit(gp, y_best, seeds=seeds, n_restarts=200)
print("Next exploit (6 d.p.):", np.round(x_next, 6))
